In [1]:
!pip install -U pip
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 42.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 41.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.40.0-py3-none-any.whl size=2531945

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import os

import nltk
nltk.download('punkt')
from nltk import word_tokenize

import string
import re

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [25]:
train_df = pd.read_csv('/content/Train.csv')
test_df = pd.read_csv('/content/Test.csv')

In [4]:
train_df

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa
...,...,...,...
5146,SW993,RAIS John Magufuli ameendelea kung’ara katika...,Kitaifa
5147,SW994,KAMPENI inayohimiza watafi ti kuandika upya h...,Kitaifa
5148,SW996,"MATUKIO mapya ya malaria kwa kila watu 1,000 ...",Kitaifa
5149,SW997,IMEELEZWA kuwa hakuna sheria yoyote inayokata...,Kitaifa


In [5]:
train_df.category.value_counts()

Kitaifa      2000
michezo      1720
Biashara     1360
Kimataifa      54
Burudani       17
Name: category, dtype: int64

In [6]:
# encoded_df = pd.get_dummies(train_df['category'])

# train_df = pd.concat([train_df, encoded_df], axis=1)
# train_df = train_df.drop(columns='category')

In [7]:
def clean_text(text):
  text = text.lower()
  text = re.sub('â€˜', '', text)
  text = re.sub('[^\w\s]', '', text)
  text = re.sub('\d+', '', text)
  text = word_tokenize(text)
  return ' '.join(text)

clean_text('SERIKALI imesema haitakuwa tayari kuona amani.')

'serikali imesema haitakuwa tayari kuona amani'

In [8]:
train_df['content'] = train_df.content.apply(clean_text)
train_df

,id,content,category
0,SW0,serikali imesema haitakuwa tayari kuona amani ...,Kitaifa
1,SW1,mkuu wa mkoa wa tabora aggrey mwanri amesitish...,Biashara
2,SW10,serikali imetoa miezi sita kwa taasisi zote za...,Kitaifa
3,SW100,kampuni ya mchezo wa kubahatisha ya mbet imein...,michezo
4,SW1000,watanzania wamekumbushwa kusherehekea sikukuu ...,Kitaifa
...,...,...,...
5146,SW993,rais john magufuli ameendelea kungara katika s...,Kitaifa
5147,SW994,kampeni inayohimiza watafi ti kuandika upya hi...,Kitaifa
5148,SW996,matukio mapya ya malaria kwa kila watu katika ...,Kitaifa
5149,SW997,imeelezwa kuwa hakuna sheria yoyote inayokataz...,Kitaifa


In [9]:
# from sklearn.model_selection import train_test_split

# X = train_df['content']
# y = train_df[['Biashara', 'Burudani', 'Kimataifa', 'Kitaifa', 'michezo']]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [10]:
df = train_df[['content', 'category']]
df_train = df.sample(frac=0.85, random_state=2020)
df_test = df.drop(df_train.index)
len(df_train), len(df_test)

(4378, 773)

In [11]:
import ktrain
from ktrain import text

MODEL_NAME = 'bert-base-multilingual-uncased'

MAXLEN  = 128
batch_size = 32
learning_rate = 5e-5
epochs = 3

In [12]:
transformer = text.Transformer(MODEL_NAME, maxlen=MAXLEN)
trn = transformer.preprocess_train(df_train.content.values, df_train.category.values)
val = transformer.preprocess_test(df_test.content.values, df_test.category.values)
model = transformer.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=batch_size)
learner.fit(learning_rate, epochs)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

preprocessing train...
language: sw
train sequence lengths:
	mean : 309
	95percentile : 663
	99percentile : 1052


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: sw
test sequence lengths:
	mean : 304
	95percentile : 614
	99percentile : 1105


Epoch 1/3
137/137 [==============================] - 146s 889ms/step - loss: 0.7172 - accuracy: 0.6921 - val_loss: 0.5193 - val_accuracy: 0.8085
Epoch 2/3
137/137 [==============================] - 121s 887ms/step - loss: 0.4480 - accuracy: 0.8413 - val_loss: 0.3932 - val_accuracy: 0.8629
Epoch 3/3
137/137 [==============================] - 123s 897ms/step - loss: 0.3401 - accuracy: 0.8737 - val_loss: 0.3994 - val_accuracy: 0.8577


In [13]:
log_loss = learner.validate(class_names=transformer.get_classes())[0]

25/25 [==============================] - 10s 272ms/step
              precision    recall  f1-score   support

    Biashara       0.78      0.84      0.81       193
    Burudani       0.50      0.25      0.33         4
   Kimataifa       0.00      0.00      0.00        10
     Kitaifa       0.84      0.80      0.82       290
     michezo       0.93      0.97      0.95       276

    accuracy                           0.86       773
   macro avg       0.61      0.57      0.58       773
weighted avg       0.85      0.86      0.85       773



In [17]:
test_df

,swahili_id,content
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,"MKUU wa Wilaya ya Bahi, Mkoani Dodoma, Mwanah..."
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,"MWISHONI mwa wiki hii, Timu ya Soka ya Taifa,..."
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,THAMANI ya mauzo ya bidhaa za Afrika Masharik...
3,00868eeee349e286303706ef0ffd851f39708d37,MENEJA Mawasiliano na Utetezi wa asasi ya AGP...
4,00a5cb12d3058dcf2e42f277eee599992db32412,"WAZIRI wa Kilimo, Japhet Hasunga amesema seri..."
...,...,...
1283,feb4b0ae88524c9cee3e50f2301d84a235f3c607,MKURUGENZI Msaidizi Msajili wa Asasi za kirai...
1284,fecf3f14f47237e02721ed4baa4eb6c11abd239f,Kilomoni alisema jana kesho atazungumzia sual...
1285,feed09e13586d12139cd59bc20996a4d29706606,"TIMU ya taifa ya soka ya Tanzania Bara, Kilim..."
1286,ffc0bca6fde8fa0cce1ac3a7b0d746603d441bf8,"YANGA imetozwa faini ya Sh 3,500,000 kutokana..."


In [23]:
test_df['cleaned_content'] = test_df['content'].apply(clean_text)

In [48]:
test_data = transformer.preprocess_test(test_df.content.values)

predictions = learner.predict(test_data)

preprocessing test...
language: sw
test sequence lengths:
	mean : 316
	95percentile : 642
	99percentile : 997


1288/1288 [==============================] - 40s 31ms/step


In [55]:
predictions

array([[1.9840756e-02, 8.9043658e-04, 6.3272542e-03, 9.5671540e-01,
        1.6226182e-02],
       [1.8785601e-03, 1.2611677e-03, 8.9040282e-04, 3.6119554e-02,
        9.5985031e-01],
       [2.1134914e-01, 1.5138983e-03, 9.1337776e-03, 7.6334786e-01,
        1.4655376e-02],
       ...,
       [1.8745728e-03, 1.3527329e-03, 9.8312658e-04, 4.3951824e-02,
        9.5183778e-01],
       [1.8789605e-03, 1.2016926e-03, 8.6924026e-04, 3.7946042e-02,
        9.5810401e-01],
       [1.5698956e-02, 2.4039128e-03, 2.4275789e-02, 9.2497873e-01,
        3.2642700e-02]], dtype=float32)

In [56]:
predicted_indices = np.argmax(predictions, axis=1)

# Define category labels
category_labels = ['Biashara', 'Burudani', 'Kimataifa', 'Kitaifa', 'michezo']

# Map the indices to the corresponding category labels
predicted_categories = [category_labels[idx] for idx in predicted_indices]

# Print the predicted categories
print(predicted_categories)

['Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Biashara', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'michezo', 'Kitaifa', 'Kitaifa', 'Biashara', 'Kitaifa', 'michezo', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'michezo', 'michezo', 'Biashara', 'Biashara', 'michezo', 'Kitaifa', 'Biashara', 'Biashara', 'michezo', 'Biashara', 'Kitaifa', 'Kitaifa', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'michezo', 'Biashara', 'Kitaifa', 'Biashara', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'michezo', 'Biashara', 'Biashara', 'Kitaifa', 'michezo', 'Biashara', 'michezo', 'Kitaifa', 'michezo', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'michez

In [57]:
results_df = pd.DataFrame({'test_id': test_df['swahili_id'], 'Content': test_df['content'], 'Prediction': predicted_categories})

In [60]:
results_df.Prediction.value_counts()

Kitaifa      483
michezo      447
Biashara     354
Kimataifa      2
Burudani       2
Name: Prediction, dtype: int64

In [32]:
val_df = train_df[:2500]
val_df

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa
...,...,...,...
2495,SW3810,MAMLAKA ya Udhibiti wa Mifuko ya Jamii (SSRA)...,Kitaifa
2496,SW3811,"MREMBO wa Tanzania, Anita Mlay ameibuka mshin...",Kitaifa
2497,SW3813,MWILI wa mvuvi aliyekufa maji baada ya mtumbw...,Kitaifa
2498,SW3816,"WAZIRI wa Afya, Maendeleo ya Jamii, Jinsia, W...",Kitaifa


In [33]:
val_df['content'] = val_df['content'].apply(clean_text)

In [34]:
val_data = transformer.preprocess_test(val_df.content.values)

val_predictions = learner.predict(val_data)

predicted_indices = np.argmax(val_predictions, axis=1)

# Define category labels
category_labels = ['Biashara', 'Burudani', 'Kimataifa', 'Kitaifa', 'michezo']

# Map the indices to the corresponding category labels
predicted_categories = [category_labels[idx] for idx in predicted_indices]

# Print the predicted categories
print(predicted_categories)

preprocessing test...
language: sw
test sequence lengths:
	mean : 313
	95percentile : 678
	99percentile : 1087


2500/2500 [==============================] - 74s 30ms/step
['Kitaifa', 'Biashara', 'Kitaifa', 'michezo', 'Kitaifa', 'michezo', 'Biashara', 'Kitaifa', 'Biashara', 'michezo', 'Biashara', 'michezo', 'Biashara', 'Biashara', 'Kitaifa', 'Biashara', 'Kitaifa', 'Biashara', 'Kitaifa', 'Biashara', 'Kitaifa', 'Biashara', 'Kitaifa', 'michezo', 'Biashara', 'michezo', 'Biashara', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'Biashara', 'Kitaifa', 'Kitaifa', 'Biashara', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'michezo', 'michezo', 'Kitaifa', 'Biashara', 'michezo', 'Kitaifa', 'michezo', 'michezo', 'Biashara', 'michezo', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'michezo', 'Kitaifa', 'Biashara', 'Biashara', 'Kitaifa', 'Kitaifa', 'michezo', 'Biashara', 'Kitaifa', 'Biashara', 'Kitaifa', 'Kitaifa', 'Kitaifa', 'Biashara', 'Kitaifa', 'Kitaifa', 'michezo', 'miche

In [35]:
val_df['predicted_labels'] = predicted_categories
val_df

,id,content,category,predicted_labels
0,SW0,serikali imesema haitakuwa tayari kuona amani ...,Kitaifa,Kitaifa
1,SW1,mkuu wa mkoa wa tabora aggrey mwanri amesitish...,Biashara,Biashara
2,SW10,serikali imetoa miezi sita kwa taasisi zote za...,Kitaifa,Kitaifa
3,SW100,kampuni ya mchezo wa kubahatisha ya mbet imein...,michezo,michezo
4,SW1000,watanzania wamekumbushwa kusherehekea sikukuu ...,Kitaifa,Kitaifa
...,...,...,...,...
2495,SW3810,mamlaka ya udhibiti wa mifuko ya jamii ssra in...,Kitaifa,Kitaifa
2496,SW3811,mrembo wa tanzania anita mlay ameibuka mshindi...,Kitaifa,michezo
2497,SW3813,mwili wa mvuvi aliyekufa maji baada ya mtumbwi...,Kitaifa,Kitaifa
2498,SW3816,waziri wa afya maendeleo ya jamii jinsia wazee...,Kitaifa,Kitaifa


In [36]:
correct_predictions = (val_df['category'] == val_df['predicted_labels']).sum()

# Calculate accuracy
accuracy = correct_predictions / len(val_df)

print("Accuracy:", accuracy)

Accuracy: 0.8868


In [61]:
encoded_df = pd.get_dummies(results_df['Prediction'])

results_df = pd.concat([results_df, encoded_df], axis=1)
results_df = results_df.drop(columns=['Prediction', 'Content'])

In [62]:
results_df

,test_id,Biashara,Burudani,Kimataifa,Kitaifa,michezo
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,0,0,0,1,0
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,0,0,0,0,1
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,0,0,0,1,0
3,00868eeee349e286303706ef0ffd851f39708d37,0,0,0,1,0
4,00a5cb12d3058dcf2e42f277eee599992db32412,1,0,0,0,0
...,...,...,...,...,...,...
1283,feb4b0ae88524c9cee3e50f2301d84a235f3c607,0,0,0,1,0
1284,fecf3f14f47237e02721ed4baa4eb6c11abd239f,0,0,0,0,1
1285,feed09e13586d12139cd59bc20996a4d29706606,0,0,0,0,1
1286,ffc0bca6fde8fa0cce1ac3a7b0d746603d441bf8,0,0,0,0,1


In [63]:
results_df.to_csv('results1.csv', index=False)